In [28]:
"""


Aim: Looking at the old data, and the new data to see the changes. 
Currently: It is being done manually.
Objective: Automate it.

Manual Process
1. Get the updated data by exporting it from PowerBI.
2. Get the old data from the All Facilities Data.
3. Copying the old data below the new data and highlighting it in green (in Excel).
4. Conditional Filtering - Duplicate Values.
5. Filter by Color on the BME#
    - No-fill means these are new equipments added
    - Green means these devices got removed. 

    Something new: Making the old and new entries in different worksheets.
    No color though.
6. Depending on the scope, add or remove it from the All Facilities Data (updating the old data file).


For example:
Old.xlsx has the following data:
There are two columns: No and Name
No: 1,2,3,4,5,6
Name: Blue, Green, Yellow, Orange, Purple, Pink

New.xlsx has the following data:
There are two columns: No and Name
No:1,2,3,7,5,6
Name:Blue,Green, Yellow, Black, Purple, Pink

The output file Highlighted.xlsx should be as follows:
There are two columns: No and Name
No:4,7
Name:Orange,Black

(4,Orange) should be highlighted yellow because they do not exist in the New.xlsx file. And (7,Black) should be highlighted blue because they do not exist in Old.xlsx file.
"""

'\n\n\nAim: Looking at the old data, and the new data to see the changes. \nCurrently: It is being done manually.\nObjective: Automate it.\n\nManual Process\n1. Get the updated data by exporting it from PowerBI.\n2. Get the old data from the All Facilities Data.\n3. Copying the old data below the new data and highlighting it in green (in Excel).\n4. Conditional Filtering - Duplicate Values.\n5. Filter by Color on the BME#\n    - No-fill means these are new equipments added\n    - Green means these devices got removed. \n\n    Something new: Making the old and new entries in different worksheets.\n    No color though.\n6. Depending on the scope, add or remove it from the All Facilities Data (updating the old data file).\n\n\nFor example:\nOld.xlsx has the following data:\nThere are two columns: No and Name\nNo: 1,2,3,4,5,6\nName: Blue, Green, Yellow, Orange, Purple, Pink\n\nNew.xlsx has the following data:\nThere are two columns: No and Name\nNo:1,2,3,7,5,6\nName:Blue,Green, Yellow, Bla

In [29]:
"""
openpyxl is a Python Library which helps us to write Excel files.

Security Issues: By default openpyxl does not guard against quadratic blowup or billion laughs xml attacks. 
To guard against these attacks install defusedxml.
"""
import openpyxl
from openpyxl.styles import PatternFill

# This is a code check to verify if the file path exists.
- Status: Works
- Improvements: None

In [42]:
#check if file exists
import os.path

path = 'C:/Users/parshaikhaa/Desktop/Automation/Script/New Data/Cobequid Community Health Centre.xlsx'

check_file = os.path.exists(path)

print(check_file)

True


# This part of the code is to extract the worksheet from All Facilities Data based on location and store it in a different worksheet.
- Status: Works
- Improvement: Trying to figure out how to do this without storing it in an intermediate file.

In [65]:
# Filter the all sites using the "Facility Name" column
# Read excel file
# Store each of the facilities in the different excel sheets - all the files are under one folder.


import pandas as pd
import os

def filter_and_store_facilities(input_file):
    # Read the Excel file
    df = pd.read_excel(input_file)

    # Group data by 'Facility Name'
    grouped = df.groupby('Facility Name')

    # Create a folder to store the output Excel files
    output_folder = "New Data"
    os.makedirs(output_folder, exist_ok=True)

    # Iterate over groups and store each group in a separate Excel file
    for facility_name, data in grouped:
        # Create an Excel writer object for this facility
        output_file = os.path.join(output_folder, f"{facility_name}.xlsx")
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            data.to_excel(writer, index=False)

# Example usage:
input_file = "C:/Users/parshaikhaa/Desktop/Automation/Script/New Data/All Facility Data Jan 2024 Export.xlsx"
filter_and_store_facilities(input_file)


# This code is used to extract the All Facility Data worksheet from the whole excel sheet and store it as new data to be compare the old data with
- Status: Works
- Note: Cobequid Community Health Centre data is present in the directory as a download.

Things to work on
- Iterating it through the New Data folder and auto-naming the new files.

In [66]:
import os
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.utils import get_column_letter

def save_worksheet_as_excel(input_file, sheet_name, output_file):
    # Load the workbook
    wb = load_workbook(input_file)
    
    # Select the desired worksheet
    ws = wb[sheet_name]
    
    # Create the directory if it doesn't exist
    output_directory = os.path.dirname(output_file)
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    # Create a new workbook and copy the contents of the selected worksheet
    new_wb = Workbook()
    new_ws = new_wb.active
    
    # Copy cell values and formatting
    for row in ws.iter_rows(values_only=True):
        new_ws.append(row)
    
    # Save the new workbook
    new_wb.save(output_file)

# Example usage:
input_file = "C:/Users/parshaikhaa/Desktop/Automation/Script/Old Data/Cobequid Community Health Centre.xlsx"
# Path to the original Excel file
sheet_name = "All Facility Data"       
# Name of the worksheet to be saved
output_file = "C:/Users/parshaikhaa/Desktop/Automation/Script/Old Data/Extract/Cobequid Community Health Centre.xlsx" # Path to the output Excel file

save_worksheet_as_excel(input_file, sheet_name, output_file)


# This code, helps to compare the differences between the two files and give us the output of the highlighted file.

- Status: Works
- Improvements: Adding labels in the output file and checking for the label column check for both the old and the new file.

Things to work on:
- Autonaming and comparing.
- Iteration

In [95]:


# Loading the data from the file
def load_data(file_path):
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active
    data = []
    for row in sheet.iter_rows(values_only=True):
        data.append(row)
    return data

# Comparing the changes between the old data and new data 
# Comparing the changes based on the "Number" column between the old data and new data
def find_changes(old_data, new_data):
    old_numbers = {row[0] for row in old_data}  # Assuming "Number" is the first column
    new_numbers = {row[0] for row in new_data}  # Assuming "Number" is the first column

    # if the record is not present in new numbers, then add to removed 
    # otherwise add it to new record list
    removed_records = [row for row in old_data if row[0] not in new_numbers]
    new_records = [row for row in new_data if row[0] not in old_numbers]

    return new_records, removed_records


# Highlighting the additions in blue and deductions in yellow - not adding the color since we are storing them on different sheets
def highlight_changes(new_records, removed_records, output_path):
    wb = openpyxl.Workbook()

    # Deleting the default sheet and adding two new worksheets of New and Old records.
    del wb["Sheet"]
    new_ws = wb.create_sheet(title="Added")
    removed_ws = wb.create_sheet(title="Removed")

    # Write new records
    for record in new_records:
        new_ws.append(record)
   
    # Write removed records
    for record in removed_records:
        removed_ws.append(record)

    # Save the workbook
    wb.save(output_path)

def main():
    old_data_path = "C:/Users/parshaikhaa/Desktop/Automation/Script/Old Data/Extract/Cobequid Community Health Centre.xlsx"
    new_data_path = "C:/Users/parshaikhaa/Desktop/Automation/Script/New Data/Cobequid Community Health Centre.xlsx"
    output_path = "Highlighted Cobequid.xlsx"

    # old_data_path = "Old.xlsx"
    # new_data_path = "New.xlsx"
    # output_path = "Highlighted.xlsx"

    old_data = load_data(old_data_path)
    new_data = load_data(new_data_path)

    new_records, removed_records = find_changes(old_data, new_data)

    highlight_changes(new_records, removed_records, output_path)

if __name__ == "__main__":
    main()
